In [8]:
import pandas as pd
import talib as TA
import pandas_ta as ta
from backtesting import Strategy, Backtest

In [ ]:
'''
This strategy takes into account RSI for entry signals and a exponential moving average for trend detection.
is intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.
The backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.
'''

READ & CLEAN THE DATA SET

In [9]:
df = pd.read_csv('data_sets/data_set_EURUSD_Candlestick_15_M_BID_31.01.2019-30.01.2022.csv')
df = df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
74758,30.01.2022 22:45:00.000 GMT-0500,1.11619,1.11629,1.11607,1.11608,709.03
74759,30.01.2022 23:00:00.000 GMT-0500,1.11609,1.11634,1.11602,1.11607,377.49
74760,30.01.2022 23:15:00.000 GMT-0500,1.11606,1.11621,1.11603,1.11615,267.24
74761,30.01.2022 23:30:00.000 GMT-0500,1.11614,1.11636,1.11608,1.11633,261.17
74762,30.01.2022 23:45:00.000 GMT-0500,1.11632,1.11636,1.11616,1.11616,242.71


CALCULATION OF ATR

In [10]:
df['ATR']= df.ta.atr()
df.tail()

,Local time,Open,High,Low,Close,Volume,ATR
74758,30.01.2022 22:45:00.000 GMT-0500,1.11619,1.11629,1.11607,1.11608,709.03,0.000465
74759,30.01.2022 23:00:00.000 GMT-0500,1.11609,1.11634,1.11602,1.11607,377.49,0.000455
74760,30.01.2022 23:15:00.000 GMT-0500,1.11606,1.11621,1.11603,1.11615,267.24,0.000435
74761,30.01.2022 23:30:00.000 GMT-0500,1.11614,1.11636,1.11608,1.11633,261.17,0.000424
74762,30.01.2022 23:45:00.000 GMT-0500,1.11632,1.11636,1.11616,1.11616,242.71,0.000408


BACKTEST ATR RELATED SL&TP

In [11]:
class MyStrat(Strategy):
    initsize = 0.3
    mysize = initsize
    ema_window = 200
    rsi_window = 3
    
    def init(self):
        super().init()
        self.ema = self.I(TA.EMA, self.data.Close, self.ema_window)
        self.rsi = self.I(TA.RSI, self.data.Close, self.rsi_window)        

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5
        price = self.data.Close[-1] 
        
        if self.position:
            self.position.close()
            
        if self.data.Close >= self.ema and self.rsi <= 10:
            sl1 = price - slatr
            tp1 = price + slatr*TPSLRatio
            
            self.buy(sl=sl1, tp=tp1, size = self.mysize)
            
        if self.data.Close <= self.ema and self.rsi >= 90:
            sl2 = price + slatr
            tp2 = price - slatr*TPSLRatio
            
            self.sell(sl=sl2, tp=tp2, size = self.mysize)

bt = Backtest(df, MyStrat, cash=500, margin=1/50, commission=.00)
stat = bt.run()
print(stat)
plot = bt.plot(plot_volume=False)
print(plot)

/tmp/ipykernel_18003/848740784.py:33: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=500, margin=1/50, commission=.00)


Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                    3.865548
Equity Final [$]                   1275.11767
Equity Peak [$]                   1310.613895
Return [%]                         155.023534
Buy & Hold Return [%]               -2.978043
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.285725
Avg. Drawdown [%]                   -0.903907
Max. Drawdown Duration                13869.0
Avg. Drawdown Duration             459.985612
# Trades                               1888.0
Win Rate [%]                         54.07839
Best Trade [%]                        0.47816
Worst Trade [%]                     -0.186084
Avg. Trade [%]                    

OPTIMIZATION

In [12]:
stats = bt.optimize(
    ema_window = [50, 100, 200],
    rsi_window = range(2, 4),
    maximize = 'Equity Final [$]',
)
print(stats)
print(stats._strategy)
bt.plot(plot_volume=False, plot_pl=False)

Backtest.optimize:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found
[18608:18659:0404/145125.955345:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[18608:18659:0404/145125.959705:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[18608:18658:0404/145126.120284:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[18608:18658:0404/145126.120405:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[18608:18658:0404/145126.503342:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[18608:18658:0404/145126.503460:ERROR:bus

Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                   13.148215
Equity Final [$]                  5034.860812
Equity Peak [$]                   5509.687085
Return [%]                         906.972162
Buy & Hold Return [%]               -2.978043
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.370547
Avg. Drawdown [%]                   -1.094701
Max. Drawdown Duration                 4344.0
Avg. Drawdown Duration             179.493261
# Trades                               6467.0
Win Rate [%]                        52.605536
Best Trade [%]                        0.47816
Worst Trade [%]                     -0.215474
Avg. Trade [%]                    

Row(id='2396', ...)

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found
[19128:19164:0404/145726.686461:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[19128:19164:0404/145726.695124:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[19128:19163:0404/145726.734751:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[19128:19163:0404/145726.734903:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[19128:19163:0404/145726.982628:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[19128:19163:0404/145726.982723:ERROR:bus